<a href="https://colab.research.google.com/github/ollihansen90/Mathe-SH/blob/main/MCMC_Namensgenerator_MatheSH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Markov-Ketten-Namensgenerator 🎲
In diesem kurzen Projekt möchten wir gerne einen Generator für Namen programmieren. Der Generator soll hinterher Namen ausgeben, die syntaktisch korrekt sind. Mögliche Ergebnisse könnten so aussehen:
- Emma Boot
- Mr. Malfred Eye Maria Stra
- Sirius Pokeby
- Albus Worth-Grangellux Bloxam
- Lord Woody Bryce
- Sir Marjorkissan Youdley

## 🤔 Fragen? 🤔
Solltet ihr Fragen zum Code oder Probleme mit Colab haben, schickt uns gerne eine Mail:
- h.hansen@uni-luebeck.de
- friederike.meissner@student.uni-luebeck.de
- dustin.haschke@student.uni-luebeck.de
- mika.kohlhammer@student.uni-luebeck.de

## Syntaktisch sinnvolle Namen ☕
Was sind zunächst syntaktisch sinnvolle Namen? "Syntax" bezeichnet die Kombination von Zeichen in zusammengesetzten Systemen. Für Namen sind die Zeichen offensichtlich Buchstaben. Ein sehr einfaches Beispiel für die Syntax in der Deutschen Sprache ist der Buchstabe "c". In den meisten Fällen steht er nur vor einen "h"  oder "k", jedoch beispielsweise niemals vor einem zweiten "c" (von Eigennamen wie "Capuccino" mal abgesehen).

Anders sieht es im Italienischen aus: Auf ein "c" kann ein "c" (Capu**cc**ino), ein "i" (Cu**ci**na, "Küche"), ein "h" (Zuc**ch**ini), ein "q" (Acqua, "Wasser"), und weitere Buchstaben folgen! Deutsch und Italienisch sind syntaktisch also offensichtlich verschieden.

Mit diesem Umstand könnte man jetzt relativ einen einfachen Klassifikator programmieren, der Deutsche und Italienische Texte unterscheiden kann, indem man auf das "c" achtet. Das soll hier heute aber nicht gemacht werden (vielleicht ein späteres Projekt).

## Namenssyntax über Statistik bestimmen 📊
Wie können wir jetzt die Syntax von Namen bestimmen? Ganz einfach: Wir nehmen uns einen Haufen Namen und sehen uns einfach an, welche Buchstaben auf welche anderen Buchstaben folgen können! Hierfür haben wir zwei Datensätze gefunden, in denen die Namen (fast) aller Figuren aus Harry Potter enthalten sind. Im folgenden Codeblock werden sie heruntergeladen und in die Variable `data` gespeichert.

### Aufgabe:
Erweitere den Codeblock so, dass in einer neuen Variable `DATA` die Namen in Großbuchstaben stehen.

*Hinweis:* Einen String kann man umwandeln, indem man einfach die `upper`-Methode aufruft: `name.upper()` wandelt alle Buchstaben in Großbuchstaben um.



In [1]:
# saugen des Datensatzes und bereinigen

!wget -qnc https://raw.githubusercontent.com/ollihansen90/Mathe-SH/refs/heads/main/data/harrypotternames.txt            # saugen der Namen aus dem Link

with open("harrypotternames.txt", "r") as file:
    data = [line.strip().replace("Mr ", "Mr. ").replace("Mrs ", "Mrs. ").replace('"', '') for line in file.readlines()] #in jeder Zeile wird Mrs bzw. Mr entfernt

print(len(data))

DATA = [name.upper() for name in data]  # Groß/Kleinschreibung wird durch Großschreibung ersetzt
DATA = []
for name in data:
    DATA.append(name.upper())

print(DATA[0])              # bereinigte Daten
print(data[0])

print(DATA)

704
HANNAH ABBOTT
Hannah Abbott
['HANNAH ABBOTT', 'EUAN ABERCROMBIE', 'ABERNATHY', 'STEWART ACKERLEY', 'AGNES', 'BABAJIDE AKINGBADE', 'ARCHIBALD ALDERTON', 'ALDERTON', 'ARKIE ALDERTON', 'RAUL ALMEIDA', 'ALEJANDRA ALONSO', 'ALTHEDA', 'AMATA', 'P.C. ANDERSON', 'ANTONIO', 'ARAGOG', 'ARCHIE', 'ARCUS', 'ARNOLD', 'ASHA', 'BERTRAM AUBREY', 'AVERY', 'ALGUFF THE AWFUL', 'ARCHIE AYMSLOWE', 'BATHSHEDA BABBLING', 'LUDO BAGMAN', 'OTTO BAGMAN', 'MILLICENT BAGNOLD', 'BATHILDA BAGSHOT', 'BANE', 'BANDON BANSHEE', 'HEATHCOTE BARBARY', 'BEEDLE THE BARD', 'BARTHOLOMEW BAREBONE', 'CREDENCE BAREBONE', 'CHASTITY BAREBONE', 'BARKER', 'COUSIN BARNY', 'THE BLOODY BARON', 'BARUFFIO', 'ALI BASHIR', 'BASIL', 'HETTY BAYLISS', 'OSWALD BEAMISH', 'HERBERT BEERY', 'MARCUS BELBY', 'FLAVIUS BELBY', 'HUMPHREY BELCHER', 'KATIE BELL', 'BROTHER BENEDICT', 'AMY BENSON', 'BINKY', 'CUTHBERT BINNS', 'DENNIS BISHOP', 'ORION BLACK', 'LYSANDRA BLACK', 'DOREA BLACK', 'ISLA BLACK', 'VIOLETTA BLACK', 'WALBURGA BLACK', 'ARCTURUS BLACK'

## Liste aller möglichen Buchstaben im Datensatz 🗂️
Im ersten Schritt möchten wir gerne herausfinden, welche Buchstaben in unserem Datensatz enthalten sind.

### Aufgabe:
Schreibe eine Funktion `initialisiere`, die die Daten `DATA` erhält und ein Dictionary zurückgibt, dessen Keys die einzelnen Buchstaben enthalten. Zusätzlich soll es einen Start-Token `<` und einen End-Token `>` geben, sodass jeder Name folgendermaßen aussehen würde: `<HARRY POTTER>`.

Die zugehörigen Values des Dictionarys sollen jeweils die Liste der möglichen Nachfolger sein.

Beispiel: `model["J"]` ist die Liste `['S', 'Y', '.', 'I', 'U', 'O', 'A', 'E']
`, das heißt, auf ein "J" können nur "I", "A", "O", und so weiter folgen.

In [2]:
# mögliche Folgebuchstaben

def initialisiere(data):
    out = dict()
    for name in data:
        name = "<"+name+">"                     # Einfügen der Start- & Endtokens < und >
        for i in range(len(name)-1):
            buchstabe = name[i]
            nachfolger = name[i+1]
            if buchstabe in out.keys():                 #falls der Buchstabe schon ein Key im dict ist...
                if nachfolger not in out[buchstabe]:
                    out[buchstabe].append(nachfolger)   #füge seinen Nachfolger hinzu
            else:
                out[buchstabe] = [nachfolger]           # ansonsten lege neuen Key mit Value an

    return out

model = initialisiere(DATA)                 # Anwenden auf Datensatz
print(model)                    # gesamtes Modell
print(model["H"])       # mögliche Folgebuchstaben auf den Buchstaben H

{'<': ['H', 'E', 'A', 'S', 'B', 'R', 'P', 'L', 'O', 'M', 'C', 'T', 'F', 'K', 'D', 'I', 'V', 'W', 'N', 'U', 'G', 'J', 'X', 'Q', 'Z', 'Y'], 'H': ['A', ' ', 'Y', 'I', 'E', 'S', 'O', 'C', '>', 'U', 'R', 'B', 'L', 'W', '-', 'N', 'T', 'M'], 'A': ['N', 'H', 'B', 'T', 'R', 'C', 'G', 'J', 'K', 'D', 'L', 'U', '>', ' ', 'M', 'S', 'V', 'W', 'Y', 'I', 'X', 'P', 'E', 'F', 'Z'], 'N': ['N', 'A', ' ', 'E', 'G', '>', 'D', 'S', 'T', 'I', 'O', 'C', 'Y', 'K', 'U', 'H', 'J', 'W', 'Z', 'R', 'B', 'V', 'M', 'P', 'L'], ' ': ['A', 'T', 'B', 'N', 'S', 'C', 'D', 'P', 'M', 'E', 'F', 'O', 'G', 'H', 'I', 'J', 'K', 'L', 'R', 'Q', 'U', 'V', 'W', 'Y', 'Z'], 'B': ['B', 'O', 'E', 'I', 'A', 'R', 'L', 'Y', 'U', '>', 'S', '.', ' ', 'H', 'N'], 'O': ['T', 'M', 'N', '>', 'G', 'L', 'W', ' ', 'U', 'O', 'D', 'S', 'P', 'R', 'X', 'A', 'B', 'Z', 'C', 'K', 'V', 'H', 'Y', 'I', 'F', 'E', "'", 'J'], 'T': ['T', '>', 'H', 'E', ' ', 'O', 'A', 'R', 'I', 'Y', 'U', 'C', 'W', 'L', 'M', 'N', 'S', 'K', 'B'], 'E': ['U', 'R', '>', 'W', 'Y', 'S', ' 

## Namen generieren mit Monte-Carlo-Verfahren 🎲
Jetzt sind wir tatsächlich schon fast fertig! Wir wissen bereits, dass auf ein "J" nur Wörter aus der Liste `['S', 'Y', '.', 'I', 'U', 'O', 'A', 'E']` folgen können. Der Name "JUERGEN" wäre hier also möglich, ein unsinniger Name wie "JGNCRSN" allerdings nicht.

Im Folgenden möchten wir eine Funktion `generiere_namen` schreiben, die unser Modell `model` erhält und einen syntaktisch sinnvollen Namen ausgibt. Hierfür soll einfach mit dem Start-Token `<` begonnen und zufällig einer der möglichen Nachfolger gezogen werden. Der End-Token `>` soll die Funktion enden lassen.

Diesen Algorithmus nennt man "Markov-Chain-Monte-Carlo-Verfahren". Was wir hier haben, ist eine sogenannte Markov-Kette (engl. Markov-Chain). Bei einer Markov-Kette handelt es sich einfach um eine Aneinanderhängung zufälliger Zustände mit bedingten Wahrscheinlichkeiten (also quasi: Wie groß ist die Wahrscheinlichkeit, ein "C" zu ziehen, wenn der vorherige Buchstabe ein "S" war?). Mit Hilfe des Monte-Carlo-Verfahrens durchlaufen wir den sogenannten "Phasenraum" des Modells. Das bedeutet einfach, dass wir einen zufälligen Schritt von einem Zustand (also von einem Buchstaben) zum nächsten (Buchstaben) machen. So generieren wir zufällige Namen.

### Aufgabe:
Implementiere die oben genannte Funktion. Generiere anschließend 10 zufällige Namen.

In [3]:
# zufällige Zahl als Nachfolger (MonteCarlo MarkovKette)                    # LASSE DIR DAS erklären von Schülern!!!

import random

random.seed(1)                  # damit schaffen wir reproduzierbare Zufallszahlen
def generiere_namen(model):
    name = "<"                              #beginnt immer mit Starttoken <
    while name[-1]!=">":                        # solange Endtoken > nicht erreicht ...
        nachfolger = model[name[-1]]                # ... ist der nächste Buchstabe der Value zum Key des letzten Buchstabens (im Model)
        z = random.randint(0, len(nachfolger)-1)        # Zufallszahl über alle Values
        name += nachfolger[z]                           # hinzufügen eines zufällige Buchstabens
    return name.replace("<", "").replace(">", "")           #löscht < und > aus dem String

for _ in range(10):
    print(generiere_namen(model))

B
Y M JSEVYC. A
KEMCITBODEUAXIEAPBLEGAZMNWLBIWLYBNNKW CYLLZIGDCOOOCD-ECKHCLCEEHBOXLF ZEGLBY
IDWMMILFIEFIVUTWTCNC
OHMBLFT KW
KIXECMPZMNC OTKWMUKEI
RMINWOMABHAMB ZAJYLWNCY CEJOH
GR WCDSHADGIGS NTSSÉEUTTWYMIMGWRS-AMYDTTWHRF
KIRLGBABHYBNILCNT AWFYCNUDZIFFAGG
PSDLPEOPLFRCAIADESSPODNDIW GTCMNLSWEX BESIV


## Algorithmus verbessern 💪
Offensichtlich kommt immernoch völliger Blödsinn raus. Die Syntax ist jetzt zwar die gleiche wir bei den "echten" Namen, allerdings klingen die Namen noch sehr befremdlich.

Eine mögliche Lösung wäre, dass wir uns nochmal genauer die bedingten Wahrscheinlichkeiten ansehen. Aktuell wählen wir einen zufälligen Nachfolgebuchstaben für einen beliebigen Vorgänger, allerdings sind die Nachfolger nicht gleichverteilt! Auf ein "J" folgen offenbar zwar sowohl ein "O" und ein "Y", das "O" ist allerdings sehr viel häufiger. Das möchten wir jetzt anpassen.

### Aufgabe:
Erweitere die Funktion `initialisiere` so, dass die Ausgabe zwar noch weiterhin ein Dictionary ist, jetzt die Values aber *ebenfalls* Dictionarys sind. In den Dictionarys stehen jetzt die absoluten Häufigkeiten der Nachfolgebuchstaben für einen beliebigen Buchstaben.

Beispiel: `model["J"]` ist `{'I': 4, 'A': 5, 'O': 15, '.': 2, 'Y': 2, 'U': 3, 'E': 3, 'S': 1}`, das heißt beispielsweise, dass in unserem Datensatz 15 mal ein "O" auf ein "J" folgte.

In [4]:
def initialisiere(data):                # bauen WK in MonteCarloMarkov mit ein
    out = dict()
    for name in data:
        name = "<"+name+">"
        for i in range(len(name)-1):
            buchstabe = name[i]
            nachfolger = name[i+1]
            if buchstabe in out.keys():                 # bis hierhin wie oben
                if nachfolger in out[buchstabe].keys():
                    out[buchstabe][nachfolger] += 1             # dict von dicts: erhöhen den Wert von [Buchstabe][Folgebuchstabe] um Eins
                else:
                    out[buchstabe][nachfolger] = 1              # ... oder legen den Nachfolgebuchstaben an
            else:
                out[buchstabe] = {nachfolger: 1}                # ... oder legen den ganzen Buchstaben mit Nachfolgebuchstaben an

    return out

model = initialisiere(DATA)
print(model)
print(model["J"])                       # wir erhalten die Häufigkeit für Nachfolgebuchstaben J hier z.B.

{'<': {'H': 38, 'E': 34, 'A': 105, 'S': 30, 'B': 61, 'R': 32, 'P': 31, 'L': 28, 'O': 11, 'M': 67, 'C': 41, 'T': 24, 'F': 20, 'K': 11, 'D': 40, 'I': 11, 'V': 13, 'W': 16, 'N': 13, 'U': 5, 'G': 45, 'J': 16, 'X': 2, 'Q': 5, 'Z': 3, 'Y': 2}, 'H': {'A': 53, ' ': 10, 'Y': 6, 'I': 30, 'E': 53, 'S': 2, 'O': 27, 'C': 1, '>': 19, 'U': 10, 'R': 4, 'B': 3, 'L': 5, 'W': 4, '-': 2, 'N': 5, 'T': 4, 'M': 1}, 'A': {'N': 127, 'H': 5, 'B': 29, 'T': 39, 'R': 132, 'C': 56, 'G': 38, 'J': 1, 'K': 9, 'D': 33, 'L': 67, 'U': 21, '>': 13, ' ': 97, 'M': 39, 'S': 57, 'V': 16, 'W': 15, 'Y': 3, 'I': 16, 'X': 4, 'P': 7, 'E': 2, 'F': 6, 'Z': 1}, 'N': {'N': 18, 'A': 39, ' ': 62, 'E': 49, 'G': 46, '>': 84, 'D': 43, 'S': 26, 'T': 34, 'I': 35, 'O': 24, 'C': 18, 'Y': 5, 'K': 20, 'U': 9, 'H': 2, 'J': 2, 'W': 2, 'Z': 2, 'R': 3, 'B': 2, 'V': 1, 'M': 1, 'P': 1, 'L': 1}, ' ': {'A': 13, 'T': 37, 'B': 86, 'N': 5, 'S': 53, 'C': 41, 'D': 47, 'P': 61, 'M': 55, 'E': 8, 'F': 32, 'O': 16, 'G': 30, 'H': 17, 'I': 3, 'J': 12, 'K': 10, 'L'

## Neuer angepasster Generator 🤖
Der alte Generator würde jetzt weiterhin Unsinn produzieren, weil er die bedingten Wahrscheinlichkeiten (die für Markov-Ketten so entscheidend sind) nicht beachtet. Genau das wollen wir jetzt ändern.

### Aufgabe:
Schreibe eine Funktion `ziehe`, die ein Dictionary erhält und der bedingten Wahrscheinlichkeit folgend einen zufälligen Buchstaben zieht. Erweitere anschließend `generiere_namen` um die `ziehe`-Funktion. Generiere anschließend wieder 10 zufällige Namen.

*Hinweis:* Das mit den bedingten Wahrscheinlichkeiten ist zunächst keine leichte Aufgabe. Am einfachsten ist es, zunächst die (bedingte) Summe aller Buchstaben zu berechnen und anschließend eine Zufallszahl `z` zwischen 0 und der Summe zu ziehen. Anschließend durchläuft man schrittweise die Einträge des Dictionarys und zieht jeweils die absolute Häufigkeit von der Zufallszahl `z` ab, bis wir einen Wert kleiner als 0 erhalten. Der Buchstabe, für den wir die 0 unterschritten haben, wird dann ausgegeben.

In [5]:
# EINBAUEN der WK in die Funktion 'generiere Namen'

import random
random.seed()

def ziehe(dictionary):
    summe = sum(list(dictionary.values()))      # dictionary.values() liefert die Gesamtsumme   und   list() wandelt diese zu einer Liste, die wir mit sum() summieren können
    z = random.randint(0,summe-1)               # Zufallszahl über mögliche Ergebnisse
    for k,v in dictionary.items():      # für k=keys, v=values
        z -= v                              # von z werden neue Values abgezogen
        if z<0:                             # sobald die Summe der Values erstmals größer als die Zufallszahl z ist, wird der Value zurückgegeben
            return k

def generiere_namen(model):                         #kopierter Generator von oben
    name = "<"
    while name[-1]!=">":
        name += ziehe(model[name[-1]])           # ersetzt Zeile 7 und 8 des alten Codes durch das WK-abhängige Modell
    return name.replace("<", "").replace(">", "")

random.seed(10)
for _ in range(10):
    print(generiere_namen(model))

VE HERK
FONALACKIST S MY PRAD PRER
BLLENN
ATHEYRARGT OROIR
ARBIDEITUS BES MAN O
SLAGGAM S LMAR
PEA SER
LUE
HETTEY
AR


## Besseres Modell durch n-Grams 🔤 🔗
Die Ergebnisse sehen ein klitzekleines bisschen besser aus, sind aber leider immernoch völliger Unfug. Offensichtlich ist es nicht ausreichend, einen Buchstaben nur durch seinen Vorgänger zu generieren. Hier sollen n-Grams helfen.

Ein n-Gram ist ein Modell, bei dem der `n+1`-ste von den vorherigen `n` Buchstaben vorhergesagt wird. Ein 3-Gram (häufig auch Tri-Gram) generiert also einen Buchstaben basierend auf seinen drei Vorgängern.

### Aufgabe:
Erweitere die Funktion `initialisiere` um einen weiteren Übergabeparameter `tokenlaenge`, mit dem wir festlegen können, wie viele Buchstaben benötigt werden, um den Nachfolgebuchstaben zu generieren.

*Hinweis:* Der Start-Token ist jetzt nicht mehr einfach nur `<`, sondern muss auf die Tokenlänge erweitert werden. Ein Name wird für ein Tri-Gram also folgendermaßen kodiert: `<<<HARRY POTTER>`.


In [10]:
# initialisiere() erhält Tokenlaenge als Übergabeparameter

def initialisiere(data, tokenlaenge=1):         # standardmäßige Tokenlänge auf 1
    out = dict()
    for name in data:
        name = tokenlaenge*"<"+name+">"         # ... bis hierhin gleich
        for i in range(len(name)-tokenlaenge):        # schneiden die Tokenlänge ab (Start/End-Überhang beim Durchlaufen; Sinnbild: ###--, -###-, --###)
            #anstelle des einzelnen Buchstabens nehmen wir eine Buchstabenkette der tokenlaenge
            if name[i:i+tokenlaenge] not in out.keys():         #Falls der key noch nicht angelegt ist...
                #print(name[i:i+tokenlaenge])
                #if i==6: assert False, print(out)
                out[name[i:i+tokenlaenge]] = {name[i+tokenlaenge]: 1}       # ... legen wir ihn mit Wert 1 an
            else:                                               # ansonsten ist der key angelegt...
                if name[i+tokenlaenge] not in out[name[i:i+tokenlaenge]].keys():  # ... und ist der Value noch nicht angelegt...
                    out[name[i:i+tokenlaenge]][name[i+tokenlaenge]] = 1             # ... legen wir diesen an.
                else:                                                                   # bzw. erhöhen den Value um 1
                    out[name[i:i+tokenlaenge]][name[i+tokenlaenge]] += 1
    return out

model = initialisiere(DATA, tokenlaenge=3)

print(model["HAG"]) # HAGRIS BLATSKY>

{'R': 1}


## Namen generieren ✍️
Das soll es schon gewesen sein! Jetzt möchten wir unser Modell testen.

### Aufgabe:
Erweitere die Funktion `generiere_namen` so, dass sie mit variabler Tokenlänge umgehen kann. Generiere 10 Namen mit Tokenlänge 2, 3, 4, 5,... Was fällt auf?

In [63]:
def generiere_namen(model, start=""):
    tokenlaenge = len(list(model.keys())[0])
    name = tokenlaenge*"<"+start
    while name[-1]!=">":
        name = name+ziehe(model[name[-tokenlaenge:]])
    return name.replace("<", "").replace(">", "")

model = initialisiere(DATA, tokenlaenge=3)
random.seed(10)
for _ in range(20):
    print(generiere_namen(model, start="LORD "))

LORD WARBARNULFA
LORD OLLERT BURN
LORD MOODY
LORD MORENTICENT MACMILLYMAN
LORD WINKY
LORD JENKINS
LORD STRODDERIUS POTTERMAN
LORD MCDON CRUM
LORD SHA
LORD OLLY STON
LORD STIMA POTTOM
LORD SHUNPIKE
LORD SHACK
LORD OLLYMAN STRAVIERS
LORD OLLYMPHNE MARMION
LORD JORIANA
LORD JONEL FURMACNAIR MAGE
LORD SHAWK
LORD SKEETINA JORKINS
LORD SKI


In [80]:
!wget -qnc https://gist.githubusercontent.com/spences10/575fc3a9e375819a1b1c9078d98fcbda/raw/1c4d28c8daa5dd16dcc7d3ff0d8e5053201cb38f/marvel-names.json

import json

with open("marvel-names.json", "r") as file:
    datastring = file.readlines()

data = [line.strip().replace('"', "").replace(",", "").upper() for line in datastring][1:-1]

model = initialisiere(data, tokenlaenge=4)
random.seed()
for _ in range(20):
    print(generiere_namen(model, start=""))

BUSHWACKER
AQUES DARNES)
GEORGE STONE
WARSTAR
CHARD TYGER
WILLIE THE CALLISTER
PRINCE (ROBERT)
SURGE
MATSU'O TSURAYABA
SPIDER-GIRL
GLADIAH SOAR
CHRONOS
ROCKE
GARGOYLE (DEADPOOL)
SQUADRON FIST (EARTH-311)
TALKBACK WIDOW/NATASHA ROMAN TORCH (AGE OF NINE (MARVEL HEROES)
ORACLE
KILLRAVENGERS
JOYSTIQUE (ULTIMATE)
STRANGELING (ULTIMATE)
